In [11]:
import json
import requests
import pandas as pd
from io import StringIO # python3; python2: BytesIO 
import boto3

#raw bucket name
bucket = 'final-bird-project-rae' # already created on S3

def req():
    country = "united_kingdom"
    page = 1
    numPages = 2
    recordings = []
    while page <=numPages :
        url = f"https://xeno-canto.org/api/2/recordings?query=cnt:{country}&page={page}"
        response = requests.get(url)
        numPages = int(response.json()["numPages"])
        recordings.extend(response.json()['recordings'])
        page+=1

        #print(page)
    
    jsonString = json.dumps({"records":recordings})
    return jsonString

jsonString  =req()


In [13]:
from io import BytesIO, StringIO
import json
import pandas as pd
import boto3
import requests



print('-----------------------i was fired ----------------------')

#s3 nuckets processed data would be saved
birdbucket = "s3://bird-project-live/"

#north england counties
north_counties = ['Cheshire', 'Cumbria', 'County Durham','Manchester', 'Lancashire', 'Merseyside', 'Northumberland', 'Tyne', 'Yorkshire','Lincolnshire']

s3_client = boto3.client('s3')

#raw bucket name
bucket = 'final-bird-project-rae' # already created on S3

#website to get counties and states names in th uk
url ='https://www.townscountiespostcodes.co.uk/counties-in-uk/'

"""
sends a get requests to the endpoint to get the html code which will be passed with pandas to extract the table. It can aslo be passed with beautifulsoup
pandas makes table easy to extract
"""
x = requests.get('https://www.townscountiespostcodes.co.uk/counties-in-uk/')

#extracts the first table on th epage into a dataframe
df = pd.read_html(x.text)[0]

#takes the necessary column
df = df[1:][['County','Country']]

#convert it to a dictionary, so we have the city name as a key and the counties under it as array
states = {k: g["County"].tolist() for k,g in df.groupby("Country")}



-----------------------i was fired ----------------------


In [20]:

#time of year
#03-> Maarch, 04->April, 05-> May and these are spring months
Spring= ['03', '04' ,'05']
Summer =['06', '07', '08']
Autumn = ['09', '10' , '11']
Winter = ['12', '01', '02']

def is_it_north(address):
    """
    checks an address and ckeck if any of the word match the counties categoriezed as north counties
    """
#     splitted_address = address.split(',')
#     temp = []
#     for j in splitted_address:
#         if j.strip() in north_counties:
#             temp.append('yes')
#             break

    if address in north_counties:# len(temp) <1:
        
        return 'NO'
    
    return 'YES'
    
#converts a string number to an integer
def reclean(hour):
    """
    This function was created as some numbers like 005 were passing through the filter
    This converts numbers like that to int and that removes the 00
    """
    try:
        correct_hour = int(hour)
    except:
        correct_hour = 5
    return str(correct_hour)
    
#get the hour form the time columns
#this column has different case, each of them was factored in 
#some had ??, date, 8am,8:00am etc
def clean_hour(hour):
    if hour.startswith('00') and len(hour)<4:
        real_hour = hour[-2:]
    elif '.' in hour or ':' in hour:
        hr = hour.split('.')[0].split(':')[0]
        if len(hr) <2:
            real_hour = '0'+hr
        else:
            real_hour = hr
    elif 'am' in hour and '.' not in hour:
        hr = hour.split('am')[0]
        if len(hr) <2:
            real_hour = '0'+hr
        else:
            real_hour = hr
    elif 'pm' in hour and '.' not in hour:
        try:
            hr = hour.split('pm')[0]
            if len(hr) <2:
                real_hour = '0'+hr
            else:
                real_hour = hr
        except:
            real_hour = '19'
    elif '-' in hour:
        real_hour = hour.split('-')[0]
    elif 'noon' in hour:
        real_hour = '15'
    elif '?' in hour:
        real_hour = '05'
    else:
        real_hour = '05'
    send_hour = reclean(real_hour)
    
    return send_hour
    
#checks if england is the address colummn
def england_or_not(address):
    if 'england' in address.lower():
        answer = 'England'
    else:
        answer ='Not England'
    return answer
    
#check which season it was using the month in the date
def check_season(daytime):
    month = daytime.split('-')[1]
    
    if month in Spring:
        season='spring'
    elif month in Summer:
        season= 'summer'
    elif month in Autumn:
        season='autumn'
    elif month in Winter:
        season='winter'
    else:
        season='No season'
    return season


#10-4am night, 4-12 morning,12-5 afternoon, 6-10 evening
def time_of_day(daytime):
    hour = daytime.split(':')
    
    try:
        if int(hour[0]) < 13 and int(hour[0])  > 3:
            answer = 'Morning'
        elif int(hour[0]) > 13 and int(hour[0])  < 18:
            answer = 'afternoon'
        else:
            answer = 'Evening'
    except:
        answer = 'NO TIME'
    return answer


#retired this function
# def hour(daytime):
#     hour = daytime.split(':')
#     return hour[0]

#gets the year from the date
def get_upload_year(daytime):
    year = daytime.split('-')[0]
    return year


#dtermine the gender by searching for male, female in the column
def gender(desc):
    desc = desc.lower()
    if 'female' in desc:
        bird_gender = 'female'
    elif 'male' in desc and 'female' not in desc:
        bird_gender= 'male'
    else:
        bird_gender= 'No gender specified'
    return bird_gender

#check if the desc contains the word song or call
def is_call_or_song(desc):
    desc = desc.lower()
    if 'song' in desc and 'call' not in desc:
        sound = 'song'
    elif 'call' in desc and 'song' not in desc:
        sound = 'call'
    elif 'call' in desc and 'song' in desc:
        sound = 'both'
    else:
        sound = 'Not specified'
    return sound


#gets the county name
def check_county(addr):
    """
    loop through the list of states in the uk
    loop through each county under them
    check if the county is in the address
    it returns the county name if found and return a default if it does find any
    """
    for i in states:
        for j in states[i]:
            if j in addr:
                return j
            
    return 'No County'
    
#gets the country
def check_country(addr):
    
    for i in states:

        if i in addr:
            return i

    return 'No Country'

#converts it to a dataframe
df = pd.json_normalize(json.loads(jsonString)['records'])
df = df.dropna()

In [21]:

#replace the columns names
df.columns = ['xeno_canto_catalogue_number','specie_generic_name','specific_name','subspecie_name','specie_eng_name','recordist_name','recording_made_in','locality_name','lat','lng','alt','rec_sound_type','url','audio_file_url','file_original_name','license_desc_url','rec_quality_rating','len_in_mins','rec_time','rec_date','upload_date','bg_specie','add_remarks','identified_visually','lure_with_playback','sono_small','sono_medium','sono_large','sono_full']


#extarct rows with england in the locality name
contains_england = df # df[df['locality_name'].str.contains('England')]

#contains_england['country'] = contains_england['locality_name'].apply(england_or_not)
contains_england['county'] = contains_england['locality_name'].apply(check_county)
contains_england['country'] = contains_england['locality_name'].apply(check_country)

#north or south england
#creates a new column with yes or no function
#contains_england['is_north'] = contains_england['locality_name'].apply(is_it_north)
contains_england['is_north'] = contains_england['county'].apply(is_it_north)

contains_england['time_of_day'] = contains_england['rec_time'].apply(time_of_day)

contains_england['specie_generic_name'] = contains_england['specie_generic_name'].str.title()
contains_england['specific_name'] = contains_england['specific_name'].str.title()
contains_england['specie_eng_name'] = contains_england['specie_eng_name'].str.title()
contains_england['recordist_name'] = contains_england['recordist_name'].str.title()
contains_england['locality_name'] = contains_england['locality_name'].str.title()
contains_england['rec_sound_type'] = contains_england['rec_sound_type'].str.title()
contains_england['add_remarks'] = contains_england['add_remarks'].str.title()



#male or female or no gender specified
contains_england['gender'] = contains_england['rec_sound_type'].apply(gender)


#contains_england['recording_hour'] =  contains_england['rec_time'].apply(hour)

contains_england['recording_hour'] =  contains_england['rec_time'].apply(clean_hour)
contains_england['season'] = contains_england['rec_date'].apply(check_season)


#convert to dateime and ignore errors
contains_england['rec_date'] = pd.to_datetime(contains_england['rec_date'],errors='coerce')
contains_england['rec_day'] = contains_england['rec_date'].dt.day.astype('Int64')
contains_england['rec_month'] = contains_england['rec_date'].dt.month.astype('Int64')
contains_england['rec_year'] = contains_england['rec_date'].dt.year.astype('Int64')



contains_england['upload_date'] = pd.to_datetime(contains_england['upload_date'],errors='coerce')
contains_england['upload_day'] = contains_england['upload_date'].dt.day.astype('Int64')
contains_england['upload_month'] = contains_england['upload_date'].dt.month.astype('Int64')
contains_england['upload_year'] = contains_england['upload_date'].dt.year.astype('Int64')

contains_england['days_diff']=(contains_england['upload_date'] - contains_england['rec_date']).dt.days
contains_england['call_song'] = contains_england['rec_sound_type'].apply(is_call_or_song)

print('----------------------droping rec date and upload datet----------------------')
contains_england.drop('rec_date',axis=1,inplace=True)

contains_england.drop('upload_date',axis=1,inplace=True)

print('----------------------selecting non empty rows----------------------')
ce = contains_england[~(contains_england['lat'].isna())]
ce = ce[(ce['lat'] !='')]
export = ce[~ce['rec_day'].isna()]
print('----------------------converting lat and long to float----------------------')
export['lat'] = export['lat'].astype(float)
export['lng'] = export['lng'].astype(float)



----------------------droping rec date and upload datet----------------------
----------------------selecting non empty rows----------------------
----------------------converting lat and long to float----------------------


C:\Users\DELL\AppData\Local\Temp/ipykernel_9600/963714466.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export['lat'] = export['lat'].astype(float)
C:\Users\DELL\AppData\Local\Temp/ipykernel_9600/963714466.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export['lng'] = export['lng'].astype(float)


In [22]:
### Is there a specie that is more frequent in the Noth England than the south of EANGLAND

In [29]:
north = export[export['is_north']=='YES']['specie_generic_name'].value_counts().reset_index()
south = export[export['is_north']!='YES']['specie_generic_name'].value_counts().reset_index()

In [34]:
both = north.merge(south, on='index', how='left')

In [39]:
both.columns

Index(['index', 'specie_generic_name_x', 'specie_generic_name_y'], dtype='object')

In [37]:
both.query('specie_generic_name_x > specie_generic_name_y').head()

,index,specie_generic_name_x,specie_generic_name_y
0,Phylloscopus,4197,130.0
1,Sylvia,3413,84.0
2,Turdus,3201,136.0
3,Emberiza,2871,80.0
4,Fringilla,1888,49.0


In [38]:
### Is there a specie that sings more frequently in the morning than in the evening

In [42]:
export['time_of_day'].unique()

array(['Morning', 'NO TIME', 'Evening', 'afternoon'], dtype=object)

In [81]:
morning = export[export['time_of_day']=='Morning']
morning_call = morning[morning['call_song']=='call'].groupby('specie_generic_name').size().reset_index(name="morning_call_count")

evening = export[export['time_of_day']=='Evening']
evening_call = evening[evening['call_song']=='call'].groupby('specie_generic_name').size().reset_index(name="evening_call_count")


In [82]:
both = morning_call.merge(evening_call, on='specie_generic_name')

In [83]:
both

,specie_generic_name,morning_call_count,evening_call_count
0,Acanthis,47,8
1,Accipiter,30,2
2,Acrocephalus,37,13
3,Actitis,20,136
4,Aegithalos,333,7
...,...,...,...
138,Troglodytes,339,13
139,Turdus,827,414
140,Tyto,14,173
141,Uria,31,3


In [84]:
both.query('morning_call_count > evening_call_count').head()

,specie_generic_name,morning_call_count,evening_call_count
0,Acanthis,47,8
1,Accipiter,30,2
2,Acrocephalus,37,13
4,Aegithalos,333,7
5,Aix,11,4


In [86]:
### Analysis of the time of the day and potentially look at different time of the year e.g Do we have more recordings in the spring than winter


In [88]:
export.columns

Index(['xeno_canto_catalogue_number', 'specie_generic_name', 'specific_name',
       'subspecie_name', 'specie_eng_name', 'recordist_name',
       'recording_made_in', 'locality_name', 'lat', 'lng', 'alt',
       'rec_sound_type', 'url', 'audio_file_url', 'file_original_name',
       'license_desc_url', 'rec_quality_rating', 'len_in_mins', 'rec_time',
       'bg_specie', 'add_remarks', 'identified_visually', 'lure_with_playback',
       'sono_small', 'sono_medium', 'sono_large', 'sono_full', 'county',
       'country', 'is_north', 'time_of_day', 'gender', 'recording_hour',
       'season', 'rec_day', 'rec_month', 'rec_year', 'upload_day',
       'upload_month', 'upload_year', 'days_diff', 'call_song'],
      dtype='object')

In [89]:
export['time_of_day'].value_counts()

Morning      35082
Evening       8184
afternoon     2151
NO TIME        954
Name: time_of_day, dtype: int64

In [90]:
export['season'].value_counts()

spring    22741
summer    11226
winter     7043
autumn     5361
Name: season, dtype: int64

In [91]:
###  Do we have more species for which we have only 1 subspecie

In [98]:
specie_subspecie_count = export[['specie_generic_name','subspecie_name']].groupby(['specie_generic_name'])['subspecie_name'].size().reset_index(name="counts")

In [105]:
only_one_subspecie = specie_subspecie_count[specie_subspecie_count['counts']==1].shape[0]

In [106]:
have_more_than_one = specie_subspecie_count[specie_subspecie_count['counts']!=1].shape[0]

In [107]:
have_more_than_one >only_one_subspecie

True

In [108]:
### what is the most frequent subspecie

In [110]:
export['subspecie_name'].value_counts().head()

             33227
trochilus     1429
communis       780
obscurus       761
coelebs        750
Name: subspecie_name, dtype: int64